# Echo-A 1B5 (Memory Finetune 2)
Fine tune 2 process, continues off with masking everything but the output.
This forcces all the loss learning only to be on the output token, and not on the instruction set / etc.

> This project assumes you have the rwkv-infctx conda env setup, and you are executing in that environment - see the main README.md for the conda env setup steps

## Optional: Download the pretrained model
(if you want to skip the the basemodel train + instruct tune)


In [1]:
# Init required dirs
!mkdir -p ../../../model/
!mkdir -p ../../../datapath/
!mkdir -p ../../../checkpoint/

# Download the Stage2.pth file
!rm -rf ../../../model/Echo-A-1B5-Tune1.pth
!cd ../../../model/ && wget https://huggingface.co/picocreator/memory-size-experiment-for-rwkv/resolve/main/Echo-A-1B5-Tune1.pth
!ls -alh ../../../model/Echo-A-1B5-Tune1.pth

## Prepare the dataset

Prepare and preload the finetuning process dataset

In [2]:
%%script bash
# Reset the dataset dir
mkdir -p ./dataset
rm -rf ./dataset/*.jsonl

# Generate the various datasets
echo "## Generating word reptition dataset ##"

# We do a strong bias for smaller word count, to teach the concept from scratch
# so that the model can learn the function, without learning new words all the time
python ./memory_script/gen_limited_prompt_completion_jsonl.py ./dataset/word-2-pc-count.jsonl  2  20000 &
python ./memory_script/gen_limited_prompt_completion_jsonl.py ./dataset/word-5-pc-count.jsonl  5  20000 &
python ./memory_script/gen_limited_prompt_completion_jsonl.py ./dataset/word-10-pc-count.jsonl 10 20000 &
python ./memory_script/gen_limited_prompt_completion_jsonl.py ./dataset/word-20-pc-count.jsonl 20 20000 &
python ./memory_script/gen_limited_prompt_completion_jsonl.py ./dataset/word-40-pc-count.jsonl 40 20000 &
python ./memory_script/gen_limited_prompt_completion_jsonl.py ./dataset/word-80-pc-count.jsonl 80 20000 &

# With a slight mix of the larger word count
python ./memory_script/gen_limited_prompt_completion_jsonl.py ./dataset/word-100-pc-count.jsonl 100 5000 &
python ./memory_script/gen_limited_prompt_completion_jsonl.py ./dataset/word-200-pc-count.jsonl 100 5000 &

wait
echo "## Done ##"

ls -alh ./dataset/

## Generating word reptition dataset ##
Generated JSONL file with - 2 max words, 20000 samples - at ./dataset/word-2-pc-count.jsonl
Generated JSONL file with - 5 max words, 20000 samples - at ./dataset/word-5-pc-count.jsonl
Generated JSONL file with - 10 max words, 20000 samples - at ./dataset/word-10-pc-count.jsonl
Generated JSONL file with - 100 max words, 5000 samples - at ./dataset/word-100-pc-count.jsonl
Generated JSONL file with - 100 max words, 5000 samples - at ./dataset/word-200-pc-count.jsonl
Generated JSONL file with - 20 max words, 20000 samples - at ./dataset/word-20-pc-count.jsonl
Generated JSONL file with - 40 max words, 20000 samples - at ./dataset/word-40-pc-count.jsonl
Generated JSONL file with - 80 max words, 20000 samples - at ./dataset/word-80-pc-count.jsonl
## Done ##


## Configure your environment settings
(!Important: you will need to rerun the below cell, if you restart your kernel)

In [3]:
DEEPSPEED_STRAT="deepspeed_stage_1"
GPU_DEVICES="auto"
ENABLE_WANDB=True
WANDB_PREFIX="Echo-A-1B5"

print("DEEPSPEED_STRAT:", DEEPSPEED_STRAT)
print("ENABLE_WANDB:", ENABLE_WANDB)
print("GPU_DEVICES:", GPU_DEVICES)

if ENABLE_WANDB:
    WANDB_MODE="online"
else:
    WANDB_MODE="disabled"

# Computing the notebook, and various paths
import os
NOTEBOOK_DIR=os.path.dirname(os.path.abspath("__file__"))
PROJECT_DIR=os.path.abspath(os.path.join(NOTEBOOK_DIR, "../../../"))
TRAINER_DIR=os.path.abspath(os.path.join(PROJECT_DIR, "./RWKV-v4neo/"))

print("NOTEBOOK_DIR:", NOTEBOOK_DIR)
print("TRAINER_DIR:", TRAINER_DIR)
print("PROJECT_DIR:", PROJECT_DIR)

DEEPSPEED_STRAT: deepspeed_stage_1
ENABLE_WANDB: True
GPU_DEVICES: auto
NOTEBOOK_DIR: /root/picocreator-memory-experiment/notebook/experiment/memory-enwiki
TRAINER_DIR: /root/picocreator-memory-experiment/RWKV-v4neo
PROJECT_DIR: /root/picocreator-memory-experiment


## Stage 3 : Simple Memory finetuning

In [4]:
# Lets preload the requried dataset (enwiki_100k)
!cd "{TRAINER_DIR}" && \
    python3 preload_datapath.py "{NOTEBOOK_DIR}/Echo-A-1B5-mem-finetune-2.yaml"

# Ensure the checkpoint directory exists
!cd "{TRAINER_DIR}" && mkdir -p "../checkpoint/Echo-A-1B5-mem-finetune-2/"

Extracting data files: 100%|█████████████████████| 1/1 [00:00<00:00, 500.99it/s]
Dataset json downloaded and prepared to /root/.cache/huggingface/datasets/json/default-335764146096c119/0.0.0/8bb11242116d547c741b2e8a1f18598ffdd40a1d4f2a2872c7a28b697434bc96. Subsequent calls will reuse this data.
100%|████████████████████████████████████████████| 1/1 [00:00<00:00, 193.31it/s]


In [5]:
# Start the foundation model training
!cd "{TRAINER_DIR}" && \
    export WANDB_MODE="{WANDB_MODE}" && \
    python lightning_trainer.py fit \
        -c "{NOTEBOOK_DIR}/Echo-A-1B5-mem-finetune-2.yaml" \
        --trainer.logger.init_args.name="{WANDB_PREFIX} - Mem-Finetune-2 (bs=256, train-ctx=512, {DEEPSPEED_STRAT})" \
        --trainer.strategy="{DEEPSPEED_STRAT}" \
        --trainer.devices="{GPU_DEVICES}"  \
        --model.ctx_len=512

Setting ds_accelerator to cuda (auto detect)
[RWKV.model] Running RWKV model using 'torch-jit' with torch '2.0.1+cu118'
/usr/local/lib/python3.11/dist-packages/lightning/fabric/utilities/seed.py:39: UserWarning: No seed found, seed set to 1106774813
  rank_zero_warn(f"No seed found, seed set to {seed}")
Global seed set to 1106774813
wandb: Currently logged in as: picocreator. Use `wandb login --relogin` to force relogin
wandb: wandb version 0.15.5 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade
wandb: Tracking run with wandb version 0.15.4
wandb: Run data is saved locally in ./wandb/run-20230712_131901-wyeni6zu
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run (8x3090) Echo-A-1B5 - Mem-Finetune-2 (bs=256, train-ctx=512, deepspeed_stage_1)
wandb: ⭐️ View project at https://wandb.ai/picocreator/RWKV-Memory-Experiment
wandb: 🚀 View run at https://wandb.ai/picocreator/RWKV-Memory-Experiment/runs/wyeni6zu
Using /root/.cache/torch_extensions/py31

In [8]:
# Lets export the model from the checkpoint
!cd "{TRAINER_DIR}" && \
    python export_checkpoint.py \
        "../checkpoint/Echo-A-1B5-mem-finetune-2/last.ckpt" \
        "../model/Echo-A-1B5-Tune2.pth"
!cd "{TRAINER_DIR}" && ls -alh ../model/Echo-A-1B5-Tune2.pth

Setting ds_accelerator to cuda (auto detect)
Processing zero checkpoint '../checkpoint/Echo-A-1B5-mem-finetune-2/last.ckpt/checkpoint'
Detected checkpoint of type zero stage ZeroStageEnum.optimizer_states, world_size: 8
Parsing checkpoint created by deepspeed==0.9.3
Reconstructed fp32 state dict with 438 params 1515106304 elements
Saving fp32 state dict to ../model/Echo-A-1B5-Tune2.pth
-rw-r--r-- 1 root root 5.7G Jul 12 15:23 ../model/Echo-A-1B5-Tune2.pth


In [7]:
# Lets do a quick dragon prompt validation
!python3 ./memory_script/eval_memory_guided.py "{PROJECT_DIR}/model/Echo-A-1B5-Tune2.pth"

Using /root/.cache/torch_extensions/py311_cu118 as PyTorch extensions root...
Detected CUDA files, patching ldflags
Emitting ninja build file /root/.cache/torch_extensions/py311_cu118/wkv_cuda/build.ninja...
Building extension module wkv_cuda...
Allowing ninja to set a default number of workers... (overridable by setting the environment variable MAX_JOBS=N)
ninja: no work to do.
Loading extension module wkv_cuda...
RWKV_JIT_ON 1 RWKV_CUDA_ON 1 RESCALE_LAYER 0

Loading /root/picocreator-memory-experiment/model/Echo-A-1B5-Tune2.pth ...
Strategy: (total 24+1=25 layers)
* cuda [float32, float32], store 25 layers
0-cuda-float32-float32 1-cuda-float32-float32 2-cuda-float32-float32 3-cuda-float32-float32 4-cuda-float32-float32 5-cuda-float32-float32 6-cuda-float32-float32 7-cuda-float32-float32 8-cuda-float32-float32 9-cuda-float32-float32 10-cuda-float32-float32 11-cuda-float32-float32 12-cuda-float32-float32 13-cuda-float32-float32 14-cuda-float32-float32 15-cuda-float32-float32 16-cuda-fl